In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Attention, GlobalAveragePooling2D
import tensorflow as tf

In [72]:
model = load_model('./trained_model.h5', compile=False)
# model.layers.pop(-1)

In [73]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 124)               127100    
                                                                 
Total params: 3,696,828
Trainable params: 3,662,716
Non-trainable params: 34,112
_________________________________________________________________


In [74]:
new_model = tf.keras.models.Sequential(model.layers[:-1])

for layer in new_model.layers:
    layer.trainable = False

In [75]:
new_model.add(Dense(6, activation='softmax', name='output'))

In [76]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 output (Dense)              (None, 6)                 6150      
                                                                 
Total params: 3,575,878
Trainable params: 6,150
Non-trainable params: 3,569,728
_________________________________________________________________


In [77]:
new_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [3]:
train_data_dir = './fine_tune_dataset/train/'
test_data_dir = './fine_tune_dataset/test/'
img_height = 224
img_width = 224
batch_size = 4

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical') # set as training data

test_generator = train_datagen.flow_from_directory(
    test_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical') # set as validation data

Found 1320 images belonging to 6 classes.
Found 6 images belonging to 6 classes.


In [80]:
history = new_model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    epochs = 60)

Epoch 1/60
330/330 [==============================] - 5s 8ms/step - loss: 3.7163 - accuracy: 0.4424
Epoch 2/60
330/330 [==============================] - 2s 7ms/step - loss: 0.7497 - accuracy: 0.8076
Epoch 3/60
330/330 [==============================] - 2s 7ms/step - loss: 0.4576 - accuracy: 0.8538
Epoch 4/60
330/330 [==============================] - 2s 7ms/step - loss: 0.3102 - accuracy: 0.8932
Epoch 5/60
330/330 [==============================] - 2s 7ms/step - loss: 0.1948 - accuracy: 0.9273
Epoch 6/60
330/330 [==============================] - 2s 7ms/step - loss: 0.1778 - accuracy: 0.9333
Epoch 7/60
330/330 [==============================] - 2s 7ms/step - loss: 0.1935 - accuracy: 0.9280
Epoch 8/60
330/330 [==============================] - 2s 7ms/step - loss: 0.1415 - accuracy: 0.9576
Epoch 9/60
330/330 [==============================] - 2s 7ms/step - loss: 0.1202 - accuracy: 0.9583
Epoch 10/60
330/330 [==============================] - 2s 7ms/step - loss: 0.1335 - accuracy: 0.9523

In [81]:
new_model.evaluate(test_generator)

2/2 [==============================] - 1s 23ms/step - loss: 0.0012 - accuracy: 1.0000


[0.0012012965744361281, 1.0]

In [82]:
import cv2

In [91]:
test_img = cv2.imread('./fine_tune_dataset/test/lekshmi/lekshmi_4.png')
test_img = cv2.resize(test_img, (224,224))
test_img = test_img/255

In [92]:
np.shape(test_img)

(224, 224, 3)

In [93]:
test_img = np.reshape(test_img, (1, 224, 224, 3))

In [94]:
preds = new_model.predict(test_img)

1/1 [==============================] - 0s 22ms/step


In [95]:
np.argmax(preds)

3

In [14]:
data = test_generator.class_indices

search_label = 0



abhirami


In [97]:
new_model.save('fine_tuned.h5')